In [ ]:
# 导入必要的模块
import csv
import os
import pymysql
import pandas as pd
import datetime
import time
from tqdm import tqdm

# 记录开始时间
time1 = time.time()

def connect_to_database():
    """连接到数据库"""
    return pymysql.connect(host='172.16.50.150', user='root', password='fz@2023&', database='input')

def execute_query(cursor, query):
    """执行数据库查询"""
    cursor.execute(query)

def insert_data(cursor, table_name, data_frame):
    """将数据插入数据库"""
    existing_columns = get_table_columns(cursor, table_name)
    valid_columns = [col for col in data_frame.columns if col in existing_columns]
    valid_columns_not_null = [col for col in valid_columns if data_frame[col].notnull().any()]

    insert_sql = f"INSERT INTO {table_name} ({', '.join(valid_columns)}) VALUES ({', '.join(['%s'] * len(valid_columns))}) ON DUPLICATE KEY UPDATE {', '.join([f'`{col}` = VALUES(`{col}`)' for col in valid_columns_not_null])};"
    values = [tuple(str(row[col]) for col in valid_columns) for index, row in data_frame.iterrows()]

    cursor.executemany(insert_sql, values)

def get_table_columns(cursor, table_name):
    """获取数据库表的列"""
    cursor.execute(f"SHOW COLUMNS FROM {table_name}")
    return [column[0] for column in cursor.fetchall()]

def main():
    """主函数"""
    # 记录开始时间
    time_start = time.time()

    # 连接到数据库
    conn = connect_to_database()

    # 查询配置表
    with conn.cursor() as c1:
        execute_query(c1, 'SELECT * FROM Oech_Analyse_J_configure')
        columns = [column[0] for column in c1.description]
        df_config = pd.DataFrame(c1.fetchall(), columns=columns)

    # 查询电厂文档表
    with conn.cursor() as c2:
        execute_query(c2, 'SELECT Power_plant_name, Unit_number, num_type, date, file_name FROM Oech_Analyse_power_plant_j_documentation WHERE is_write != "是" OR is_write IS NULL')
        table_data1 = c2.fetchall()

    # 构造文件路径列表
    full_paths = [os.path.join('Z:/power_plant_data_j', *row[:-1], row[-1]).replace('\\', '/') for row in table_data1]
    table_data2 = [row + (path,) for row, path in zip(table_data1, full_paths)]

    # 构建列名替换字典
    header_replacement_dict = {value: column for column, values in df_config.items() for value in values}

    with conn.cursor() as cursor:
        # 遍历文件数据列表
        for a in tqdm(table_data2, desc='处理文件', unit='文件'):
            try:
                with open(a[-1], 'r', encoding='utf-8') as csvfile:
                    reader = csv.reader(csvfile)
                    rows = [row for index, row in enumerate(reader) if index not in [0, 2]]

                columns2 = rows[0]
                data_rows = [row[:-1] for row in rows[1:]]
                new_data = pd.DataFrame(data_rows, columns=columns2)

                start_time = new_data.iloc[:, 0].iloc[0]
                end_time = new_data.iloc[:, 0].iloc[-1]
                second_time = new_data.iloc[:, 0].iloc[1]

                start_time = datetime.datetime.strptime(start_time.strip(), "%Y/%m/%d %H:%M:%S")
                second_time = datetime.datetime.strptime(second_time.strip(), "%Y/%m/%d %H:%M:%S")
                end_time = datetime.datetime.strptime(end_time.strip(), "%Y/%m/%d %H:%M:%S")

                data_granularity = second_time - start_time
                data_granularity = format(data_granularity)[-10:]

                new_data.columns = [header_replacement_dict[col] if col in header_replacement_dict else col for col in new_data.columns]

                new_data['power_plant_id'] = a[0][:3]
                new_data['power_plant_name'] = a[0]
                new_data['unit_number'] = a[1]
                new_data['num_type'] = a[2]
                new_data['data_granularity'] = data_granularity

                valid_data = new_data.loc[:, ~new_data.columns.duplicated()]

                for table_name in ['Oech_Analyse_J_VIB', 'Oech_Analyse_J_TCA_FGH', 'Oech_Analyse_J_Operational_history', 'Oech_Analyse_J_LUBE_OIL', 'Oech_Analyse_J_FUEL_GAS', 'Oech_Analyse_J_ECA', 'Oech_Analyse_J_DTC', 'Oech_Analyse_J_CPFM', 'Oech_Analyse_J_CONTROL', 'Oech_Analyse_J_BPT', 'Oech_Analyse_J_BEARING', 'Oech_Analyse_J_AIR']:
                    insert_data(cursor, table_name, valid_data)

                update_sql = f"UPDATE Oech_Analyse_power_plant_j_documentation SET is_write = '是' WHERE Power_plant_name = '{a[0]}' AND Unit_number = '{a[1]}' AND file_name = '{a[4]}';"
                cursor.execute(update_sql)
                conn.commit()

            except FileNotFoundError:
                tqdm.write(f"文件未找到: {a[-1]} 跳过下一个文件。")
                continue

            except Exception as e:
                tqdm.write(f"处理文件时出错 {a[-1]}: {e}")
                continue

    # 创建数据库连接
    connection = connect_to_database()
    cursor = connection.cursor()

    # 构造插入语句
    insert_query = f"""
    INSERT INTO Oech_Analyse_data_j_granularity ( date_part, power_plant_id, unit_number, num_type, min_time_part, max_time_part, data_granularity ) 
    SELECT
    date_part,
    power_plant_id,
    unit_number,
    num_type,
    min_time_part,
    max_time_part,
    data_granularity 
    FROM
        (
            WITH RankedData AS (
            SELECT DISTINCT DATE
                ( TIME ) AS date_part,
                TIME ( TIME ) AS time_part,
                power_plant_id,
                unit_number,
                num_type,
                data_granularity,
                ROW_NUMBER() OVER (
                    PARTITION BY DATE ( TIME ),
                    power_plant_id,
                    unit_number,
                    num_type,
                    data_granularity 
                ORDER BY
                TIME ( TIME )) AS rn_asc,
                ROW_NUMBER() OVER ( PARTITION BY DATE ( TIME ), power_plant_id, unit_number, num_type, data_granularity ORDER BY TIME ( TIME ) DESC ) 
                AS rn_desc 
            FROM
                Oech_Analyse_J_Operational_history 
            ) SELECT
            power_plant_id,
            unit_number,
            num_type,
            date_part,
            MIN( CASE WHEN rn_asc = 1 THEN time_part END ) AS min_time_part,
            MAX( CASE WHEN rn_desc = 1 THEN time_part END ) AS max_time_part,
            data_granularity 
        FROM
            RankedData 
        GROUP BY
            date_part,
            power_plant_id,
            unit_number,
            num_type,
            data_granularity 
        ) AS Result 
    ON DUPLICATE KEY UPDATE min_time_part = Result.min_time_part,
    max_time_part = Result.max_time_part,
    data_granularity = Result.data_granularity;
    """

    cursor.execute(insert_query)

    connection.commit()
    cursor.close()
    connection.close()

    # 记录结束时间
    time_end = time.time()
    time_sum = time_end - time_start
    print(f"执行时间: {time_sum} 秒")

# 主程序入口
if __name__ == "__main__":
    main()

# 记录结束时间
time2 = time.time()
time_sum = time2 - time1
print(f"执行时间: {time_sum} 秒")

time.sleep(10)

# 关闭cmd窗口
os.system("taskkill /f /im cmd.exe")
